In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
tree = ET.parse('Stats.xml')
root = tree.getroot()
songscores = root.find('SongScores')

## Load Dataframes

In [ ]:
# You can fill this with any packs you don't want to show in the stats.
# Use cases: Packs that have been removed from the cab (but still appear in the Stats.xml)
IGNORED_PACKS = set([
])

In [ ]:
# construct played songs dataframe

def allsongnames(songscores):
    def strip_prefix(string, prefix):
        if string.startswith(prefix):
            return string[len(prefix):]
        return string
    
    names = []
    highscores = []
    for song in songscores:
        songdir = song.get('Dir') 
        
        # note: need to additionally deal with "AdditionalSongs" paths
        # (packs from AdditionalSongFolders will show as `AdditionalSongs/pack/song/` instead of `pack/song/`)
        # solution(?): strip everything but the last two segments of the path
        # not sure if AdditionalSongs is the only case this will happen,
        # but hopefully this handles anything else that might show up?
        parts = songdir.strip('/').split('/')
        *_, pack, songname = parts
        songdir = f'{pack}/{songname}/'
        
        if pack in IGNORED_PACKS:
            continue
        
        editcount = 0
        for steps in song.findall('Steps'):
            steptype = steps.get('StepsType')
            difficulty = steps.get('Difficulty')
            # if there are multiple edits, give them unique names to make processing easier, "Edit", "Edit-1", "Edit-2", etc.
            if difficulty == 'Edit':
                if editcount >= 1:
                    difficulty = f'{difficulty}-{editcount}'
                editcount += 1
            numplayed = int(steps.find('HighScoreList/NumTimesPlayed').text)
            lastplayed = pd.Timestamp(steps.find('HighScoreList/LastPlayed').text)
            names.append((songdir, steptype, difficulty, numplayed, lastplayed))
            
            # don't track leaderboards for USB customs, leads to rank inflation
            if pack != '@mem':
                leaderboards = []
                for score in steps.find('HighScoreList').findall('HighScore'):
                    # don't include any scores on slower ratemods
                    # (might be better to clean this up in the stats file itself!)
                    modifiers = score.find('Modifiers').text
                    if 'xMusic' in modifiers:
                        mods = modifiers.split(',')
                        ratemod = next(i for i in mods if 'xMusic' in i)
                        ratemod = ratemod.strip().replace('xMusic', '')
                        ratemod = float(ratemod)
                        if ratemod < 1:
                            continue
                        
                    leaderboards.append((score.find('Name').text, float(score.find('PercentDP').text)))

                leaderboards.sort(key=lambda x: x[1], reverse=True)
                for i,v in enumerate(leaderboards):
                    highscores.append((songdir, steptype, difficulty, i+1, v[0], v[1]))
                
    return names, highscores

playcount, highscores = allsongnames(songscores)

df_playedsongs = pd.DataFrame(playcount, columns=['key', 'steptype', 'difficulty', 'playcount', 'lastplayed'])
df_playedsongs = df_playedsongs.set_index(['key', 'steptype', 'difficulty'])

df_leaderboards = pd.DataFrame(highscores, columns=['key', 'steptype', 'difficulty', 'place', 'player', 'dp'])
df_leaderboards = df_leaderboards.set_index(['key', 'steptype', 'difficulty'])

In [ ]:
# load availablesongs csv as a dataframe

import os
from pathlib import Path
import csv
from collections import Counter

AVAILABLE_SONGS_PATH = 'available.csv'

def loadfromcsv(path):
    with open(path, newline='', encoding='utf8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        return [row for row in reader]

try:
    availablesongs = loadfromcsv(AVAILABLE_SONGS_PATH)
except FileNotFoundError:
    display(f"Error: couldn't load {AVAILABLE_SONGS_PATH}. Report data may be incomplete")
    availablesongs = []
    
data = []
encountered = Counter()
for i,x in enumerate(availablesongs):
    # implement IGNORED_PACKS list
    pack,songname = x[0].strip('/').split('/')
    if pack in IGNORED_PACKS:
        continue
    
    # if a duplicate difficulty is encountered, name it "Edit", "Edit-1", Edit-2", ...
    key = (x[0], x[2], x[3])
    if key in encountered:
        availablesongs[i][3] = f'{availablesongs[i][3]}-{encountered[key]}'
    availablesongs[i][4] = int(availablesongs[i][4])
    encountered[key] += 1
    data.append(availablesongs[i])

df_availablesongs = pd.DataFrame(data, columns=['key', 'songname', 'steptype', 'difficulty', 'meter'])
df_availablesongs = df_availablesongs.set_index(['key', 'steptype', 'difficulty'])

In [ ]:
df_availablesongs

In [ ]:
# combine the dataframes together
combined = df_playedsongs.combine_first(df_availablesongs)
combined['playcount'] = combined['playcount'].fillna(0).astype(int)

# my own personal editing: remove any entries for songs which don't appear in the available songs list
# if this is commented out, then entries for packs that have been removed/updated will appear in the final list
# which may make the chart count and pack completion calculations inaccurate
# note that this also removes the @mem pack
# combined = combined[combined.index.isin(df_availablesongs.index)]

# store the songname column for later, when calculating the pack_info dataframe
# drop it from the combined frame cause I don't want the data in this location
songnames = combined['songname']
combined = combined.drop(columns=['songname'])
combined

In [ ]:
# sort index for aesthetics (e.g. difficulties show up in Easy, Medium, Hard, Challenge order)
def pdict(arr):
    return {v:k for k,v in enumerate(arr)}

MODE = pdict(['dance-single', 'dance-double', 'pump-single', 'pump-double'])
DIFFS = pdict(['Beginner', 'Easy', 'Medium', 'Hard', 'Challenge'])

def sorter_difficulty_spread(s):
    if s.name == 'steptype':
        return s.map(lambda x: MODE.get(x, len(MODE)))
    elif s.name == 'difficulty':
        return s.map(lambda x: DIFFS.get(x, len(DIFFS)))
    return s

combined = combined.sort_index(key=sorter_difficulty_spread)

In [ ]:
# construct the pack_info dataframe: pack and song name for each key
data = []
for k,songname in songnames.groupby('key').first().items():
    parts = k.strip('/').split('/')
    pack, *_, inferred_songname = parts
    # if songname data doesn't exist, fall back to inferring the song name from the folder name
    if songname is None:
        songname = inferred_songname
    data.append((k, pack, songname))

pack_info = pd.DataFrame(data, columns=['key', 'pack', 'song']).set_index(['key'])
pack_info

In [ ]:
v = combined.join(pack_info)

steptypes = {'dance-single': 'S', 'dance-double': 'D'}
steptypes_full = {'dance-single': 'Single', 'dance-double': 'Double'}
difficulties = {'Beginner': 'B', 'Easy': 'E', 'Medium': 'M', 'Hard': 'H', 'Challenge': 'X', 'Edit': 'Z'}

def shorthand(row):
    # SB, SE, SM, SH, SX
    # (song name) SX10
    # idea (not implemented): display edit name? (song name) SZ69 iunno
    steptype = row.name[1]
    diff = row.name[2].partition('-')[0]
    s = steptypes.get(steptype, None)
    d = difficulties.get(diff, None)
    if s is None or d is None:
        return None
    sfull = steptypes_full.get(steptype, None)
    meter = '' if pd.isna(row.meter) else int(row.meter)
    dtag = f'{s}{d}{meter}'
    return (f'{row.song} {dtag}', dtag, sfull)

song_shorthand = v.apply(shorthand, axis=1, result_type='expand')
song_shorthand = song_shorthand.rename(columns=dict(enumerate(['shorthand', 'dtag', 'stepfull'])))
song_shorthand

In [ ]:
# get a list of ddr songs on the cab, used in some calculations
v = combined.join(pack_info)
ddr_song_list = v[v.pack.str.contains("DDR") | v.pack.str.contains("DanceDanceRevolution")].index
ddr_song_list

In [ ]:
# function that should be used in the analysis stages to grab data
# does caching so it's more efficient than calling combined.join(pack_info) all the time...

import functools

@functools.cache
def get_song_data(with_songinfo=False, with_mem=False):
    global combined, pack_info
    df = combined
    if not with_mem:
        not_including_mem = df.join(pack_info)
        not_including_mem = not_including_mem[not_including_mem.pack != '@mem']
        df = df.loc[not_including_mem.index]
    if with_songinfo:
        df = df.join(pack_info)
    return df

## Helper Queries

In [ ]:
# find a particular song name
pack_info[pack_info.song.str.contains('The Game')]

In [ ]:
# get leaderboards for a given song
df_leaderboards.loc['In The Groove/The Game/']

In [ ]:
# get played songs for a given pack
c = get_song_data(with_songinfo=True)
c[(c.pack == "In The Groove") & (c.playcount > 0)]

## Data Analysis

### General

In [ ]:
# chart and song counts for each pack (total, singles, doubles)
data = get_song_data(with_songinfo = True)

def group_by_songs(df):
    return df[~df.index.get_level_values('key').duplicated()]

def songs_and_charts(v, prefix=''):
    if prefix != '':
        prefix = f'{prefix}_'    
    total_charts = v.groupby('pack').size().rename(f'{prefix}charts')
    total_songs = group_by_songs(v).groupby('pack').size().rename(f'{prefix}songs')
    return total_songs, total_charts

# note: there might be other chart types, like pump-single, pump-double, or weird ones like lights-cabinet
# to avoid counting unplayable stuff, I guess we'll filter "total charts" to only count dance-single and dance-double
normal = songs_and_charts(data.loc[pd.IndexSlice[:, ['dance-single', 'dance-double'], :]])
singles = songs_and_charts(data.loc[pd.IndexSlice[:, ['dance-single'], :]], 'singles')
doubles = songs_and_charts(data.loc[pd.IndexSlice[:, ['dance-double'], :]], 'doubles')

v = pd.concat([*normal, *singles, *doubles], axis=1).fillna(0).sort_index(key=lambda s: s.str.lower()).reset_index()
display(v)
v.to_clipboard(index=False, header=False)

In [ ]:
# difficulty histogram

DIFFICULTY_LIMIT = 27  # group together anything above this block number, handles any joke diffs "69", 420", "9001"

# for the purposes of the histogram, label any charts with NaN meter (charts in Stats.xml but not found in song folder) as meter 0
# (if we leave as NaN, any charts with NaN meter will be ignored by the < operator)
normal = data[data.meter.fillna(0) < DIFFICULTY_LIMIT].groupby(['pack', 'meter']).size().to_frame()
above = (
    data[data.meter >= DIFFICULTY_LIMIT].groupby(['pack']).size().to_frame()
    .assign(meter=DIFFICULTY_LIMIT).set_index('meter', append=True)
)
total = normal.combine_first(above)
normalized = total / total.groupby('pack').max()
histogram = normalized.unstack().sort_index(key=lambda s: s.str.lower())
display(histogram)
histogram.to_clipboard(index=False, header=False)

### Most Played Charts

In [ ]:
def make_most_played_charts(combined):
    most_played_charts = combined.sort_values('playcount', ascending=False).head(50)
    
    # Pack / Song / Steptype (Singles / Doubles) / Difficulty (Expert) / Meter (9) / Playcount / Last played
    a = (
        most_played_charts
        .join(song_shorthand).join(pack_info)
        .reset_index(level='difficulty')
        [['pack', 'song', 'stepfull', 'difficulty', 'meter', 'playcount', 'lastplayed']]
    )
    return a

a = make_most_played_charts(get_song_data())
display(a)
a.to_clipboard(index=False, header=False)

### Most Played Charts - Doubles

In [ ]:
a = make_most_played_charts(get_song_data().loc[pd.IndexSlice[:, ['dance-double'], :]])
display(a)
a.to_clipboard(index=False, header=False)

### Most Played Songs

In [ ]:
def make_most_played_songs(combined):
    # get list of songs with most plays on them
    playcount_sum = (
        combined.groupby(level="key")
        .agg({'playcount': 'sum', 'lastplayed': 'max'})
        .rename(columns={'playcount': 'total'})
        .sort_values('total', ascending=False)
        .head(50)
    )

    # get playcount breakdown for each song in playcount_sum
    playcount_breakdown = (
        combined.loc[playcount_sum.index.values, 'playcount']
        .unstack(level=[1, 2])  # move stepstype,difficulty to columns
    )
    
    # make sure each difficulty has a BEMHX difficulty spread
    for mode in playcount_breakdown.columns.get_level_values(0).unique().to_list():
        for diff in ['Beginner', 'Easy', 'Medium', 'Hard', 'Challenge']:
            if (mode, diff) not in playcount_breakdown.columns:
                playcount_breakdown[(mode, diff)] = float('nan')
    
    # resort columns
    playcount_breakdown = playcount_breakdown.sort_index(key=sorter_difficulty_spread, axis=1)

    # join the tables together
    playcount_sum.columns = pd.MultiIndex.from_product([playcount_sum.columns, ['']])
    playcount_breakdown = playcount_sum.join(playcount_breakdown)
    
    # format for display
    v = pack_info.copy(deep=False)
    v.columns = pd.MultiIndex.from_product([pack_info.columns, ['']])
    v = v.join(playcount_breakdown, how='right')

    return v
    
v = make_most_played_songs(get_song_data())
display(v)
v.to_clipboard(index=False, header=False)

#### Most Played Songs - Doubles

In [ ]:
v = make_most_played_songs(get_song_data().loc[pd.IndexSlice[:, ['dance-double'], :]])
display(v)
v.to_clipboard(index=False, header=False)

### Most played packs + charts within pack

In [ ]:
v = get_song_data(with_songinfo=True)

# generate the top N most played songs in each pack
place = v.sort_values('playcount', ascending=False).groupby('pack').cumcount() + 1
x = v.assign(place=place)
x = x[x.place <= 10].join(song_shorthand)
y = x.reset_index().set_index(['pack', 'place'])

# unstack N most played songs + some column reordering
def most_played_songs_by_pack_sorter(s):
    if s.name == 'place':
        return s
    else:
        ordering = ['shorthand', 'playcount']
        return s.map({k:v for v,k in enumerate(ordering)})

def a(s):
    return "({playcount}) {shorthand}".format(**s)

most_played_songs_by_pack = (
    y[['shorthand', 'playcount']]
    .apply(a, axis=1)
    .unstack('place')
    # .unstack('place').reorder_levels([1, 0], axis=1).sort_index(axis=1, key=most_played_songs_by_pack_sorter)
    # pd.concat([most_played_packs], axis=1, keys=[''])
)

# calculate the most played packs
most_played_packs = (
    v.groupby('pack')
    .agg({'playcount': 'sum', 'lastplayed': 'max'})
    .sort_values(by='playcount', ascending=False)
)

# format for display
v = most_played_packs.join(most_played_songs_by_pack).reset_index()
display(v)
v.to_clipboard(index=False, header=False)

### Recently Played Packs

In [ ]:
last_played_packs = (
    get_song_data().groupby(level='key')
    .agg({'lastplayed': 'max'})
    .join(pack_info)
    .groupby('pack')
    .agg({'lastplayed': 'max'})
    .sort_values(by='lastplayed', ascending=False)
)
v = last_played_packs.reset_index()
display(v)
v.to_clipboard(index=False, header=False)

### Pack Completion

In [ ]:
v = get_song_data(with_songinfo = True)
v_played = v[v.playcount > 0]

played_charts = v_played.groupby('pack').size()
total_charts = v.groupby('pack').size()

def group_by_songs(df):
    return df[~df.index.get_level_values('key').duplicated()]

played_songs = group_by_songs(v_played).groupby('pack').size()
total_songs = group_by_songs(v).groupby('pack').size()

In [ ]:
percentage_played = (
    pd.DataFrame(index=total_charts.index)
    .assign(
        played_songs = played_songs,
        total_songs = total_songs,
        ratio_songs = played_songs/total_songs,
        played_charts = played_charts,
        total_charts = total_charts,
        ratio_charts = played_charts/total_charts,
    )
    .fillna(0)
    .sort_values(['ratio_songs', 'total_songs'], ascending=False)
)
percentage_played

In [ ]:
GRADE_LOOKUP = {
    100: '☆☆☆☆',
    99: '☆☆☆',
    98: '☆☆',
    96: '☆',
    90: '90',
    80: '80',
    70: '70',
    60: '60',
    50: '50',
    0: '0',
}

# massage GRADE_LOOKUP into values for pd.cut
c = list(GRADE_LOOKUP.items())
c.sort(key=lambda x: x[0])
values = [i[0]/100 for i in c] + [np.inf]
labels = [i[1] for i in c]

# compute the grade of each top score in the leaderboard
top_scores_per_song = df_leaderboards[~df_leaderboards.index.duplicated(keep='first')]
grades = pd.cut(
    top_scores_per_song['dp'],
    values,
    labels=labels,
    right=False
).rename('grade')
    
grade_breakdown_per_pack = v_played.join(grades).groupby(['pack', 'grade']).size().unstack().sort_index(axis=1, ascending=False)

# format for output
v = percentage_played.join(grade_breakdown_per_pack).reset_index()
display(v)
v.to_clipboard(index=False, header=False)

### Highest Passes

In [ ]:
t = df_leaderboards.join(get_song_data(with_songinfo=True)).join(song_shorthand)

# remove DDR songs since it uses different difficulty blocks
t = t[~t.index.isin(ddr_song_list)]

# remove any charts above a certain difficulty (joke charts)
t = t[t.meter <= 27]

t = t[t.player != '4199']

t = t.sort_values(by=['meter', 'dp'], ascending=False)

def make_highest_passes(t):
    return t.head(50).reset_index()[['pack', 'song', 'stepfull', 'difficulty', 'meter', 'player', 'dp']]

In [ ]:
v = make_highest_passes(t[t.stepfull == 'Single'])
display(v)
v.to_clipboard(index=False, header=False)

#### Doubles Only

In [ ]:
v = make_highest_passes(t[t.stepfull == 'Double'])
display(v)
v.to_clipboard(index=False, header=False)

#### Pass counts

In [ ]:
t.groupby('meter').size()

### Highest Scores

In [ ]:
scores = df_leaderboards.join(get_song_data(with_songinfo=True)).sort_values(by=['dp', 'meter'], ascending=False)

def make_highest_scores(scores):
    # Pack / Song / Mode / Difficulty / Meter / Player / Score
    return (
        scores.head(100)
        .join(song_shorthand)
        # index gets reordered after joining... sort again
        .sort_values(by=['dp', 'meter'], ascending=False)
        .reset_index()
        [['pack', 'song', 'stepfull', 'difficulty', 'meter', 'player', 'dp']]
    )

In [ ]:
v = make_highest_scores(scores.loc[pd.IndexSlice[:, ['dance-single'], :]])
display(v)
v.to_clipboard(index=False, header=False)

#### Doubles Only

In [ ]:
v = make_highest_scores(scores.loc[pd.IndexSlice[:, ['dance-double'], :]])
display(v)
v.to_clipboard(index=False, header=False)

### Leaderboards

In [ ]:
# player places on leaderboard

a = df_leaderboards.groupby(['player', 'place']).size()

player_songs = df_leaderboards.groupby('player').size()
player_first_places = a.unstack().sort_values(by=1, ascending=False)
player_first_places

In [ ]:
# player "points": in a leaderboard with 4 spots, 1st place gets 4 points, 2nd place 3, 3rd place 2, 4th place 1

b = df_leaderboards.groupby(level=['key', 'steptype', 'difficulty']).size().rename('total')
with_points = df_leaderboards.join(b)
with_points['points'] = with_points['total'] - with_points['place'] + 1

player_points = with_points.groupby('player')['points'].sum()
player_points

In [ ]:
v = pd.concat([player_first_places, player_songs.rename('songs'), player_points], axis=1)
# sort by number of first places
v = v.sort_values(by=1, ascending=False)
v = v.head(100)
v = v.reset_index()

display(v)
v.to_clipboard(index=False, header=False)